In [31]:
import json

with open("./data/zones.json") as f:
    zones_data = json.load(f)

with open("./data/crops.json") as f:
    crops_data = json.load(f)

with open("./data/climate_profiles.json") as f:
    climate_data = json.load(f)



In [32]:
class Zone:
    def __init__(self, zone_id, data, climate):
        self.id = zone_id
        self.name = data["name"]
        self.soil_types = data["soil_types"]
        self.dominant_soil = data["dominant_soil"]
        self.viable_crops = data["viable_crops"]
        self.agriculture_type = data["agriculture_type"]
        self.farmers = []
        self.climate_profile = climate[zone_id]["climate_profile"]

    def add_farmer(self, farmer):
        self.farmers.append(farmer)
    
    def step(self):
        # Farmers decide crops
        for farmer in self.farmers:
            farmer.decide_crop()
        # Farmers compute yields
        for farmer in self.farmers:
            farmer.compute_yield()
        # Knowledge sharing (if group type)
        self.share_knowledge()
    
    def share_knowledge(self):
        # Only shared group exchanges information
        shared_farmers = [f for f in self.farmers if f.strategy_type=="SHARED"]
        if not shared_farmers:
            return
        # Compute average yield per crop
        crop_stats = {}
        for farmer in shared_farmers:
            crop = farmer.chosen_crop
            if crop not in crop_stats:
                crop_stats[crop] = []
            crop_stats[crop].append(farmer.yield_t_ha)
        self.shared_crop_yield = {c: sum(yields)/len(yields) for c, yields in crop_stats.items()}


In [33]:
class FarmerAgent:
    def __init__(self, id, zone, land_size, strategy_type):
        self.id = id
        self.zone = zone
        self.land_size = land_size
        self.strategy_type = strategy_type
        self.memory = []  
        self.chosen_crop = None
        self.yield_t_ha = 0
        self.profit = 0

    def decide_crop(self):
        zone = self.zone
        # Compute crop suitability scores
        suitability_scores = {}
        for crop in zone.viable_crops:
            suitability_scores[crop] = self.compute_suitability(crop)
        
        # If SHARED, adjust scores with shared knowledge
        if self.strategy_type == "SHARED" and hasattr(zone, "shared_crop_yield"):
            for crop in suitability_scores:
                if crop in zone.shared_crop_yield:
                    # combine ecological suitability and peer performance
                    suitability_scores[crop] = 0.5 * suitability_scores[crop] + \
                                               0.5 * (zone.shared_crop_yield[crop] / max(zone.shared_crop_yield.values()))
        
        # Choose crop with highest score
        self.chosen_crop = max(suitability_scores, key=suitability_scores.get)

    def compute_suitability(self, crop_name):
        # Simplified suitability: compare climate and crop requirements
        crop = crops_data["crops"][crop_name]
        # For now, just check if zone in ideal zones
        score = 1 if self.zone.id in crop["ideal_zones"] else 0.5
        # You can expand with temp/precip matching later
        return score

    def compute_yield(self):
        # Base yield * suitability (simplified)
        base_yield = crops_data["crops"][self.chosen_crop]["base_yield_t_ha"]
        # For now, yield is base yield adjusted by land size and randomness
        import random
        self.yield_t_ha = base_yield * (0.8 + 0.4 * random.random())  # ±20% variation
        self.profit = self.yield_t_ha * self.land_size  # simplified
        self.memory.append({"crop": self.chosen_crop, "yield": self.yield_t_ha, "profit": self.profit})


In [34]:
class FarmModel:
    def __init__(self, zones_data, climate_data):
        self.zones = {}
        self.farmers = []
        # Initialize zones
        for zone_id, zone_info in zones_data["zones"].items():
            self.zones[zone_id] = Zone(zone_id, zone_info, climate_data["zones"])
    
    def add_farmer(self, farmer, zone_id):
        zone = self.zones[zone_id]
        farmer.zone = zone
        zone.add_farmer(farmer)
        self.farmers.append(farmer)

    def step(self):
        for zone in self.zones.values():
            zone.step()


In [35]:
model = FarmModel(zones_data, climate_data)

# Create farmers
for i in range(25):
    model.add_farmer(FarmerAgent(i, None, 10, "INDIVIDUAL"), "SUB_HUMID")
for i in range(25,50):
    model.add_farmer(FarmerAgent(i, None, 10, "SHARED"), "SUB_HUMID")

# Run for 10 seasons
for season in range(10):
    model.step()

# Compare results
individual_yields = [f.memory[-1]["yield"] for f in model.farmers if f.strategy_type=="INDIVIDUAL"]
shared_yields = [f.memory[-1]["yield"] for f in model.farmers if f.strategy_type=="SHARED"]

print("Individual average yield:", sum(individual_yields)/len(individual_yields))
print("Shared average yield:", sum(shared_yields)/len(shared_yields))


KeyError: 'sugarcane'